In [ ]:
# --- 1. Libraries ----------------------------------------------------
import math
from typing import Dict
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.gridspec as gridspec
from matplotlib.patches import Patch
from matplotlib import colors
import matplotlib.ticker as mticker
import squarify    
from pywaffle import Waffle
from matplotlib.cm import ScalarMappable
import geopandas as gpd
import cartopy.crs as ccrs
from highlight_text import fig_text
from pypalettes import create_cmap
from pyfonts import load_google_font


In [ ]:
# --- 2. File Paths & Column Mappings ---------------------------------

# File names
NEW_BUILD_CSV: str          = "../Data/Neubauwohnungen_2015-2024.csv"
TAXABLE_INCOME_CSV: str     = "../Data/Median-Einkommen-1999-2022.csv"
EQUIVALIZED_INCOME_CSV: str = "../Data/Haushaltsäquivalenz-Einkommen_2013-2022.csv"
POPULATION_CSV: str         = "../Data/Bevölkerung_1999-2024.csv"
HOUSING_STOCK_CSV: str      = "../Data/Bestand_Wohnungen_Zürich_2008-2024.csv"
BUILDINGS_CSV: str          = "../Data/Gebäude_Zürich_1901-2024.csv"

# GeoJSON for districts
KREISE_GEOJSON: str         = "../geo_data/kreise_zuerich.geojson"

# Column mapping for new residential buildings
NEW_BUILD_COLMAP: Dict[str, str] = {
    "KreisLang": "district_name",
    "QuarLang": "quarter_name",
    "FuenfjahresPeriode_noDM": "five_year_period",
    "AnzRentner": "num_retirees",
    "AnzKinder": "num_children",
    "Wohnflaeche": "living_area",
    "AnzZimmerLevel2Cd_noDM": "num_rooms",
    "AnzWhgStat": "num_apartments",
    "AnzBestWir": "population_total",
    "MietwohnungSort": "is_rental_apartment",
}

# Column mapping for median taxable income
TAXABLE_INCOME_COLMAP: Dict[str, str] = {
    "KreisLang": "district_name",
    "StichtagDatJahr": "year",
    "KreisSort": "district_sort",
    "SteuerEinkommen_p50": "taxable_income_p50",
    "SteuerEinkommen_p25": "taxable_income_p25",
    "SteuerEinkommen_p75": "taxable_income_p75",
}

# Column mapping for equivalized income
EQUIVALIZED_INCOME_COLMAP: Dict[str, str] = {
    "KreisLang": "district_name",
    "KreisSort": "district_sort",
    "StichtagDatJahr": "year",
    "aequEK_p50": "equivalized_income_p50",
}

# Column mapping for housing stock by construction period
HOUSING_STOCK_COLMAP: Dict[str, str] = {
    "KreisLang": "district_name",
    "KreisSort": "district_sort",
    "BauperiodeLevel1Lang": "building_period",
    "BauperiodeLevel1Sort": "building_period_sort",
    "AnzZimmerLevel1Cd_noDM": "num_rooms",
    "AnzWhgStat": "num_apartments",
    "Wohnflaeche": "living_area",
}

# Column mapping for population
POPULATION_COLMAP: Dict[str, str] = {
    "StichtagDatJahr": "year",
    "KreisSort": "district_sort",
    "AnzBestWir": "population_total",
}

# Column mapping for buildings
BUILDINGS_COLMAP: Dict[str, str] = {
    "StichtagDatJahr": "year",
    "AnzNeubauGbd_noDM": "num_new_buildings",
}


def load_and_rename(csv_path: str, colmap: Dict[str, str]) -> pd.DataFrame:
    """Load a CSV file and rename its columns according to the provided mapping."""
    df = pd.read_csv(csv_path)
    return df.rename(columns=colmap)


In [ ]:
# --- 3. Data Wrangling -----------------------------------------------

# 3.1 New Buildings by District / Quarter / 5-Year Period --------------

def aggregate_new_build_by_district(
    csv_path: str = NEW_BUILD_CSV
) -> pd.DataFrame:
    """
    Aggregate new residential construction data by district and 5-year period.

    Steps:
    - Load the raw dataset and rename columns using `NEW_BUILD_COLMAP`.
    - Group by `district_name` and `five_year_period`.
    - Aggregate:
        * Summed totals for retirees, children, living area, rooms,
          apartments, and population.
        * Count of rental apartments (`is_rental_apartment == 1`).

    Parameters
    ----------
    csv_path : str
        Path to the CSV file containing new building data.

    Returns
    -------
    pd.DataFrame
        Aggregated metrics per district and 5-year period.
    """
    df = load_and_rename(csv_path, NEW_BUILD_COLMAP)

    aggregated = (
        df.groupby(["district_name", "five_year_period"])
          .agg(
              num_retirees=("num_retirees", "sum"),
              num_children=("num_children", "sum"),
              living_area=("living_area", "sum"),
              num_rooms=("num_rooms", "sum"),
              num_apartments=("num_apartments", "sum"),
              population_total=("population_total", "sum"),
              rental_apartments=("is_rental_apartment", lambda s: (s == 1).sum()),
          )
          .reset_index()
    )
    return aggregated


# 3.2 Income by District (Averaged 2015–2019) --------------------------

def aggregate_taxable_income_by_district(
    csv_path: str = TAXABLE_INCOME_CSV,
    year_min_inclusive: int = 2015,
    year_max_exclusive: int = 2019,
) -> pd.DataFrame:
    """
    Compute average taxable median income per district over a multi-year period.

    Steps:
    - Load and rename columns using `TAXABLE_INCOME_COLMAP`.
    - Filter rows to years in the interval `[year_min_inclusive, year_max_exclusive)`.
    - Group by `district_name` and `district_sort`.
    - Aggregate:
        * `median_taxable_income`: Mean of yearly taxable income medians.

    Returns
    -------
    pd.DataFrame
        District-level average taxable income.
    """
    df = load_and_rename(csv_path, TAXABLE_INCOME_COLMAP)
    df = df[(df["year"] >= year_min_inclusive) & (df["year"] < year_max_exclusive)]

    aggregated = (
        df.groupby(["district_name", "district_sort"])
          .agg(median_taxable_income=("taxable_income_p50", "mean"))
          .reset_index()
    )
    return aggregated


def aggregate_equivalized_income_by_district(
    csv_path: str = EQUIVALIZED_INCOME_CSV,
    year_min_inclusive: int = 2015,
    year_max_exclusive: int = 2019,
) -> pd.DataFrame:
    """
    Compute average equivalized median income per district over a multi-year period.

    Steps:
    - Load and rename columns using `EQUIVALIZED_INCOME_COLMAP`.
    - Filter rows to years in `[year_min_inclusive, year_max_exclusive)`.
    - Group by `district_name` and `district_sort`.
    - Aggregate:
        * `median_equivalized_income`: Mean of yearly equivalized medians.

    Returns
    -------
    pd.DataFrame
        District-level average equivalized income.
    """
    df = load_and_rename(csv_path, EQUIVALIZED_INCOME_COLMAP)
    df = df[(df["year"] >= year_min_inclusive) & (df["year"] < year_max_exclusive)]

    aggregated = (
        df.groupby(["district_name", "district_sort"])
          .agg(median_equivalized_income=("equivalized_income_p50", "mean"))
          .reset_index()
    )
    return aggregated


def build_join_df(
    taxable_income_by_district: pd.DataFrame,
    new_build_by_district: pd.DataFrame,
    equivalized_income_by_district: pd.DataFrame,
) -> pd.DataFrame:
    """
    Join taxable and equivalized income statistics with new building metrics.

    Steps:
    - First merge taxable income with new building aggregates on `district_name`
      (left join keeps all income districts).
    - Then merge equivalized income using both `district_name` and `district_sort`
      (right join keeps all districts from the previous step).

    Returns
    -------
    pd.DataFrame
        Combined dataset containing income and construction indicators per district.
    """
    join_left = pd.merge(
        taxable_income_by_district,
        new_build_by_district,
        on="district_name",
        how="left",
    )

    joined = pd.merge(
        equivalized_income_by_district,
        join_left,
        on=["district_name", "district_sort"],
        how="right",
    )
    return joined


# 3.3 Housing Stock by Building Period --------------------------------

def prepare_housing_stock_by_period(
    csv_path: str = HOUSING_STOCK_CSV
) -> pd.DataFrame:
    """
    Prepare housing stock data by building period, including normalized indicators.

    Steps:
    - Load CSV and rename columns using `HOUSING_STOCK_COLMAP`.
    - Group by district and building period.
    - Aggregate:
        * Total rooms, apartments, and living area.
    - Compute district-internal shares:
        * `percentage_apartments`: Share of apartments per building period.
        * `percentage_area`: Share of living area per building period.
    - Compute structural indicators:
        * `area_per_apartment`
        * `rooms_per_apartment`
    - Remove building period category with `building_period_sort == 1`.
    - Standardize key ratios using z-scores.

    Returns
    -------
    pd.DataFrame
        Housing stock metrics per district and building period.
    """
    df = pd.read_csv(csv_path)
    df = df[list(HOUSING_STOCK_COLMAP.keys())].rename(columns=HOUSING_STOCK_COLMAP)

    # Aggregation by district & building period
    df = (
        df.groupby(
            ["district_name", "district_sort", "building_period", "building_period_sort"]
        )
        .agg(
            num_rooms=("num_rooms", "sum"),
            num_apartments=("num_apartments", "sum"),
            living_area=("living_area", "sum"),
        )
        .reset_index()
    )

    # Normalized shares per district
    df["percentage_apartments"] = (
        df["num_apartments"] /
        df.groupby("district_name")["num_apartments"].transform("sum")
    )

    df["percentage_area"] = (
        df["living_area"] /
        df.groupby("district_name")["living_area"].transform("sum")
    )

    # Ratios
    df["area_per_apartment"] = df["living_area"] / df["num_apartments"]
    df["rooms_per_apartment"] = df["num_rooms"] / df["num_apartments"]

    # Drop undesired category
    df = df[df["building_period_sort"] != 1]

    # z-scores
    df["rooms_per_apartment_z"] = (
        df["rooms_per_apartment"] - df["rooms_per_apartment"].mean()
    ) / df["rooms_per_apartment"].std()

    df["area_per_apartment_z"] = (
        df["area_per_apartment"] - df["area_per_apartment"].mean()
    ) / df["area_per_apartment"].std()

    return df


# 3.4 Bubble Plot Data: Population + Income + Buildings ----------------

def prepare_income_population_buildings_for_bubble() -> pd.DataFrame:
    """
    Prepare combined population, income, and construction data for a bubble chart.

    Steps:
    - Load population, taxable income, and building data with their respective COLMAPs.
    - Aggregate income:
        * Group by year and district and compute median values
          (p50, p25, p75).
    - Merge population with income on year/district.
    - Aggregate buildings:
        * Group by year and sum `num_new_buildings`.
    - Merge the results so each row includes population, income, and new building counts.

    Returns
    -------
    pd.DataFrame
        Yearly district-level dataset suitable for bubble plotting.
    """
    pop = load_and_rename(POPULATION_CSV, POPULATION_COLMAP)
    income = load_and_rename(TAXABLE_INCOME_CSV, TAXABLE_INCOME_COLMAP)
    buildings = load_and_rename(BUILDINGS_CSV, BUILDINGS_COLMAP)

    # Income per year and district (median)
    income_year_kreis = (
        income[["year", "district_sort", "taxable_income_p50",
                "taxable_income_p25", "taxable_income_p75"]]
        .groupby(["year", "district_sort"])
        .median()
        .reset_index()
    )

    pop_income = pd.merge(
        pop,
        income_year_kreis,
        on=["year", "district_sort"],
        how="inner",
    )

    buildings_year = (
        buildings.groupby("year")["num_new_buildings"].sum().reset_index()
    )

    result = pd.merge(
        pop_income,
        buildings_year,
        on="year",
        how="left",
    )

    return result


# 3.5 Data loading -----------------------------------------------------

new_build_by_district = aggregate_new_build_by_district()

taxable_income_by_district     = aggregate_taxable_income_by_district()
equivalized_income_by_district = aggregate_equivalized_income_by_district()

joined_df = build_join_df(
    taxable_income_by_district,
    new_build_by_district,
    equivalized_income_by_district,
)

housing_period_df = prepare_housing_stock_by_period()
bubble_df         = prepare_income_population_buildings_for_bubble()


In [ ]:
# --- 4. Colors & Global Styling --------------------------------------

# central color palette
BLUE        = "#3D85F7"
BLUE_LIGHT  = "#5490FF"
PINK        = "#C32E5A"
PINK_LIGHT  = "#D34068"
GREEN       = "#2E9E67"
GREEN_LIGHT = "#3FB77D"

GREY40      = "#666666"
GREY25      = "#404040"
GREY20      = "#333333"
BACKGROUND  = "#F5F4EF"

# Colors for districts and quarters
COLOR_DISTRICTS = sns.color_palette("tab20", 12)
COLOR_QUARTERS  = sns.color_palette("Set3", 20)

# Global Matplotlib styling
plt.rcParams.update({
    "figure.facecolor": BACKGROUND,
    "axes.facecolor": BACKGROUND,
    "font.family": "Segoe UI",
    "font.size": 12,
    "axes.titlesize": 18,
    "axes.labelsize": 13,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "legend.fontsize": 11,
    "text.color": GREY20,
    "axes.labelcolor": GREY20,
    "axes.titlecolor": GREY20,
    "xtick.color": GREY20,
    "ytick.color": GREY20,
    "axes.edgecolor": BACKGROUND,
})

# Extra fonts for maps & specialized plots
regular_font = load_google_font("Roboto")
bold_font    = load_google_font("Roboto", weight="bold")


In [ ]:
# Vis 1: Evolution of Building in Zurich by District --------------------

df_3 = housing_period_df.copy()

# Districts in numerical order
districts_sorted = sorted(
    df_3["district_name"].unique(),
    key=lambda x: int(x.split()[-1]) 
)

x_vals = np.sort(df_3["building_period_sort"].unique())

fig, axes = plt.subplots(3, 4, figsize=(20, 12))
axes = axes.flatten()
fig.subplots_adjust(hspace=0.35)

line_rooms = plt.Line2D([], [], color=BLUE, label="Rooms per Apartment (z-Score)")
line_area  = plt.Line2D([], [], color=PINK, label="Area per Apartment (z-Score)")

for ax, district in zip(axes, districts_sorted):
    df = df_3[df_3["district_name"] == district]

    x = df["building_period_sort"].values
    y_rooms = df["rooms_per_apartment_z"].values
    y_area  = df["area_per_apartment_z"].values

    ax.plot(x, y_rooms, color=BLUE)
    ax.plot(x, y_area,  color=PINK)

    ax.fill_between(
        x, y_rooms, y_area,
        where=(y_rooms > y_area),
        interpolate=True,
        color=BLUE_LIGHT,
        alpha=0.3,
    )
    ax.fill_between(
        x, y_rooms, y_area,
        where=(y_rooms <= y_area),
        interpolate=True,
        color=PINK_LIGHT,
        alpha=0.3,
    )

    ax.set_xticks(x_vals)
    ax.set_ylim(-2, 5.5)

    ax.set_facecolor(BACKGROUND)
    for spine in ax.spines.values():
        spine.set_visible(False)

    ax.grid(which="both", lw=0.6, alpha=0.4)
    ax.tick_params(axis="both", length=0)

    ax.set_title(district, weight="bold", font=bold_font, color=GREY20)

fig.set_facecolor(BACKGROUND)

fig.text(
    0.5, 0.9,
    "Evolution of Building in Zurich by District",
    ha="center", va="top",
    fontsize=30, fontweight="bold", font=bold_font, color="#000000",
)

fig.text(
    -0.01, 0.5, "z-Score",
    va="center", rotation="vertical",
    fontsize=20, fontweight="bold", font=bold_font, color=GREY20,
)

fig.text(
    0.5, 0.06, "Construction Period (Sort Index)",
    ha="center", va="center",
    fontsize=20, fontweight="bold", font=bold_font, color=GREY20,
)

fig.legend(
    handles=[line_rooms, line_area],
    loc="lower center",
    ncol=2,
    frameon=False,
    fontsize=14,
)

plt.tight_layout(rect=[0, 0.08, 1, 0.85])

fig.savefig(
    "../Plots/Line.png",
    dpi=1000,
    bbox_inches="tight",
)


In [ ]:
# --- Vis 2: Map of Buildings in Zurich (Example: 1893–1899) -----------

proj = ccrs.Mercator()

# Custom blue color gradient
cmap_blue = create_cmap(
    colors=[
        "#E8F1FF",
        "#C9E1FF",
        "#A7CEFF",
        "#7EB5FF",
        "#559BFF",
        "#2D7FE6",
        "#1A5FBF",
    ],
    cmap_type="continuous",
    name="BlueSoft",
)

# GeoData (districts)
gdf = gpd.read_file(KREISE_GEOJSON)

# Building period sort = 2 (≈ 1893–1899)
df_old = housing_period_df[housing_period_df["building_period_sort"] == 2].copy()
df_old = df_old[["district_name", "num_apartments", "district_sort"]]

gdf_old = gdf.merge(
    df_old,
    left_on="bezeichnung",
    right_on="district_name",
    how="inner",
)

# Label adjustment: rename "Kreis X" → "District X"
gdf_old["district_name_label"] = gdf_old["district_name"].str.replace("Kreis ", "District ")

fig, ax = plt.subplots(
    figsize=(6, 4),
    dpi=300,
    subplot_kw={"projection": proj},
)

fig.set_facecolor(BACKGROUND)
ax.set_facecolor(BACKGROUND)
ax.set_axis_off()

# Title
fig_text(
    x=0.5, y=0.95,
    s="Buildings in Zurich City",
    color="black",
    fontsize=12,
    fontproperties=bold_font,
    ha="center", va="top",
    alpha=0.95,
)

fig_text(
    x=0.5, y=0.905,
    s="1893 – 1899",
    color="#1A5FBF",
    fontsize=8,
    fontproperties=bold_font,
    ha="center", va="top",
)

# Map
gdf_old.plot(
    ax=ax,
    column="num_apartments",
    cmap=cmap_blue,
    linewidth=0.6,
    edgecolor="#ffffff",
)

# Label offsets
LABEL_DY_VALUE = 0.0015
LABEL_DY_NAME  = -0.0012

# District number + name
for _, row in gdf_old.iterrows():
    rep = row.geometry.representative_point()
    x, y = rep.x, rep.y

    name = row["district_name_label"]
    num  = f"{row['num_apartments']:,}".replace(",", "'")

    x_num = x
    x_name = x

    # Manual adjustments for visual overlap
    if name in ["District 4", "District 5"]:
        x_num  = x - 0.0015
        x_name = x + 0.002

    ax.text(
        x_num, y + LABEL_DY_VALUE,
        num,
        ha="center", va="center",
        fontproperties=regular_font,
        fontsize=4.5,
        color="black",
        alpha=0.95,
        zorder=10,
    )

    ax.text(
        x_name, y + LABEL_DY_NAME,
        name,
        ha="center", va="center",
        fontproperties=regular_font,
        fontsize=3.5,
        color="black",
        alpha=0.75,
        zorder=10,
    )

# Helper for thousand separators
def thousands_formatter(x, pos):
    if x == 0:
        return "0"
    return f"{int(x):,}".replace(",", "'")

# KPI block (total apartments)
total_buildings = int(gdf_old["num_apartments"].sum())
kpi_str = thousands_formatter(total_buildings, None)

fig_text(
    x=0.5, y=0.035,
    s=kpi_str,
    color="#1A5FBF",
    fontsize=12,
    fontproperties=bold_font,
    ha="center", va="top",
)

fig_text(
    x=0.5, y=0.075,
    s="Total Buildings",
    color="black",
    fontsize=10,
    fontproperties=bold_font,
    ha="center", va="top",
)

# Colorbar
cax = fig.add_axes([0.3, 0.175, 0.01, 0.2])

vmin = gdf_old["num_apartments"].min()
vmax = gdf_old["num_apartments"].max()

sm = ScalarMappable(
    cmap=cmap_blue,
    norm=colors.Normalize(vmin=vmin, vmax=vmax),
)
sm._A = []

cbar = fig.colorbar(sm, cax=cax, orientation="vertical", pad=0.02)

cbar.ax.tick_params(labelsize=5, width=0.3, length=2, pad=1)
cbar.outline.set_linewidth(0.3)
cbar.outline.set_alpha(0.7)
cbar.ax.yaxis.set_major_formatter(mticker.FuncFormatter(thousands_formatter))

# Colorbar font
for lbl in cbar.ax.get_yticklabels():
    lbl.set_fontproperties(regular_font)

# ==============================
# Bar Plot
# ==============================

bar_data = gdf_old.sort_values("district_sort")

districts = bar_data["district_sort"].astype(str).values
values = bar_data["num_apartments"].values

norm = colors.Normalize(vmin=values.min(), vmax=values.max())
bar_colors = [cmap_blue(norm(v)) for v in values]

bar_ax = fig.add_axes([0.51, 0.185, 0.2, 0.1])
bar_ax.set_facecolor("none")

bars = bar_ax.bar(
    x=range(len(values)),
    height=values,
    color=bar_colors,
)

bar_ax.set_ylim(0, max(values) * 1.15)
bar_ax.axhline(0, linewidth=0.3, color="#444444")

# X-Ticks
xtick_fp = regular_font.copy()
xtick_fp.set_size(5)

bar_ax.set_xticks(range(len(values)))
bar_ax.set_xticklabels(districts, fontproperties=xtick_fp)

bar_ax.set_xlabel(
    "District",
    fontproperties=regular_font,
    fontsize=6,
    labelpad=2,
    color="#1A5FBF",
)

bar_ax.tick_params(axis="x", width=0.3, pad=1)
bar_ax.tick_params(axis="y", labelsize=4, pad=1, width=0.3)

# Y-Axis Font
for lbl in cbar.ax.get_yticklabels():
    lbl.set_fontproperties(regular_font)
    lbl.set_fontsize(5)

bar_ax.yaxis.tick_right()
bar_ax.yaxis.set_major_formatter(mticker.FuncFormatter(thousands_formatter))

# Spines
for spine_name, spine in bar_ax.spines.items():
    if spine_name in ["bottom", "right"]:
        spine.set_visible(True)
        spine.set_linewidth(0.3)
        spine.set_edgecolor("#444444")
        spine.set_alpha(0.7)
    else:
        spine.set_visible(False)

plt.tight_layout()

plt.savefig(
    "../Plots/Map.png",
    dpi=1000,
    bbox_inches="tight",
)


In [ ]:
# Vis 3: Growth 2020–2024 by District --------------------------

filtered = joined_df[joined_df["five_year_period"] == "2020–2024"].copy()

df_z = filtered[
    ["district_name", "district_sort",
     "living_area", "num_apartments", "population_total"]
].dropna()

df_z["district_nr"] = df_z["district_name"].str.extract(r"(\d+)").astype(int)
df_z = df_z.sort_values("district_nr")

x = df_z["district_name"]

living_abs    = df_z["living_area"] / 1000
dwellings_abs = df_z["num_apartments"]
pop_abs       = df_z["population_total"]

df_dwell_sorted  = df_z.sort_values("num_apartments", ascending=False)
df_living_sorted = df_z.sort_values("living_area", ascending=False)
df_pop_sorted    = df_z.sort_values("population_total", ascending=False)

x_dwell = df_dwell_sorted["district_name"]
x_liv   = df_living_sorted["district_name"]
x_pop   = df_pop_sorted["district_name"]

dwellings_abs_sorted = df_dwell_sorted["num_apartments"].to_numpy()
living_abs_sorted    = df_living_sorted["living_area"].to_numpy() / 1000
pop_abs_sorted       = df_pop_sorted["population_total"].to_numpy()

fig1 = plt.figure(figsize=(26, 13))
fig1.patch.set_facecolor(BACKGROUND)

fig1.suptitle(
    "Absolute Growth of New Residential Units in Zurich by District (2020–2024)",
    color="#1A1A1A",
    fontsize=30,
    fontproperties=bold_font,
    y=0.99,
)

gs = gridspec.GridSpec(3, 3, figure=fig1)

ax_abs_dwell  = fig1.add_subplot(gs[0, 0])
ax_abs_living = fig1.add_subplot(gs[0, 1])
ax_abs_pop    = fig1.add_subplot(gs[0, 2])

ax_pct  = fig1.add_subplot(gs[1, :])
ax_diff = fig1.add_subplot(gs[2, :], sharex=ax_pct)


def format_axis(ax, is_abs_plot=False):
    ax.set_facecolor(BACKGROUND)
    if is_abs_plot:
        ax.grid(False)
        ax.tick_params(axis="y", length=0)
        ax.set_yticklabels([])
    else:
        ax.grid(axis="y", alpha=0.8, color="#BEBEBE", linewidth=0.8)
    ax.set_axisbelow(True)
    for spine_name, spine in ax.spines.items():
        if spine_name == "bottom":
            spine.set_visible(True)
            spine.set_linewidth(1.2)
            spine.set_color("#BEBEBE")
        else:
            spine.set_visible(False)


# --- Absolute growth: number of apartments ---
bars_dwell = ax_abs_dwell.bar(
    x_dwell, dwellings_abs_sorted,
    color=BLUE, edgecolor=BACKGROUND, linewidth=0.7,
)
ax_abs_dwell.set_title(
    "Increase in Number of Apartments",
    pad=18,
    fontproperties=bold_font,
)
ax_abs_dwell.set_ylabel(
    "Number of Apartments",
    labelpad=25,
    fontproperties=regular_font,
)
ax_abs_dwell.bar_label(
    bars_dwell,
    padding=3,
    fmt="%.0f",
    fontsize=10,
    fontproperties=regular_font,
)
ax_abs_dwell.tick_params(axis="x", labelrotation=45)
ax_abs_dwell.set_ylim(0, dwellings_abs_sorted.max() * 1.15)
format_axis(ax_abs_dwell, is_abs_plot=True)

# Tick fonts
for lbl in ax_abs_dwell.get_xticklabels():
    lbl.set_fontproperties(regular_font)
for lbl in ax_abs_dwell.get_yticklabels():
    lbl.set_fontproperties(regular_font)


# --- Absolute growth: living area ---
bars_living = ax_abs_living.bar(
    x_liv, living_abs_sorted,
    color=PINK, edgecolor=BACKGROUND, linewidth=0.7,
)
ax_abs_living.set_title(
    "Increase in Living Area",
    pad=18,
    fontproperties=bold_font,
)
ax_abs_living.set_ylabel(
    "Living Area (×1000 m²)",
    labelpad=18,
    fontproperties=regular_font,
)
ax_abs_living.bar_label(
    bars_living,
    padding=3,
    fmt="%.0f",
    fontsize=10,
    fontproperties=regular_font,
)
ax_abs_living.tick_params(axis="x", labelrotation=45)
ax_abs_living.set_ylim(0, living_abs_sorted.max() * 1.15)
format_axis(ax_abs_living, is_abs_plot=True)

for lbl in ax_abs_living.get_xticklabels():
    lbl.set_fontproperties(regular_font)
for lbl in ax_abs_living.get_yticklabels():
    lbl.set_fontproperties(regular_font)


# --- Absolute growth: population ---
bars_pop = ax_abs_pop.bar(
    x_pop, pop_abs_sorted,
    color=GREEN, edgecolor=BACKGROUND, linewidth=0.7,
)
ax_abs_pop.set_title(
    "Increase in Population",
    pad=18,
    fontproperties=bold_font,
)
ax_abs_pop.set_ylabel(
    "Population",
    labelpad=18,
    fontproperties=regular_font,
)
ax_abs_pop.bar_label(
    bars_pop,
    padding=3,
    fmt="%.0f",
    fontsize=10,
    fontproperties=regular_font,
)
ax_abs_pop.tick_params(axis="x", labelrotation=45)
ax_abs_pop.set_ylim(0, pop_abs_sorted.max() * 1.15)
format_axis(ax_abs_pop, is_abs_plot=True)

for lbl in ax_abs_pop.get_xticklabels():
    lbl.set_fontproperties(regular_font)
for lbl in ax_abs_pop.get_yticklabels():
    lbl.set_fontproperties(regular_font)


# --- Percentage shares ---
y_living_pct = living_abs / living_abs.sum() * 100
y_dwell_pct  = dwellings_abs / dwellings_abs.sum() * 100
y_pop_pct    = pop_abs / pop_abs.sum() * 100

x_pos = np.arange(len(x))
w = 0.25

bars_y2 = ax_pct.bar(
    x_pos - w,
    y_dwell_pct,
    w,
    color=BLUE,
    edgecolor=BACKGROUND,
    linewidth=0.7,
    label="Number of Apartments (%)",
)
bars_y1 = ax_pct.bar(
    x_pos,
    y_living_pct,
    w,
    color=PINK,
    edgecolor=BACKGROUND,
    linewidth=0.7,
    label="Living Area (%)",
)
bars_y3 = ax_pct.bar(
    x_pos + w,
    y_pop_pct,
    w,
    color=GREEN,
    edgecolor=BACKGROUND,
    linewidth=0.7,
    label="Population (%)",
)

ax_pct.set_title(
    "Percentage Comparison of Shares in Total Growth",
    pad=10,
    fontproperties=bold_font,
)
ax_pct.set_ylabel(
    "Share (%)",
    labelpad=14,
    fontproperties=regular_font,
)
ax_pct.set_xticks(x_pos)
ax_pct.set_xticklabels(
    x,
    ha="center",
    rotation=45,
    fontproperties=regular_font,
)

ax_pct.legend(
    loc="upper right",
    frameon=True,
    facecolor="white",
    prop=regular_font,
)

ax_pct.bar_label(
    bars_y2,
    padding=3,
    fmt="%.1f",
    fontsize=9,
    fontproperties=regular_font,
)
ax_pct.bar_label(
    bars_y1,
    padding=3,
    fmt="%.1f",
    fontsize=9,
    fontproperties=regular_font,
)
ax_pct.bar_label(
    bars_y3,
    padding=3,
    fmt="%.1f",
    fontsize=9,
    fontproperties=regular_font,
)
ax_pct.set_ylim(0, 30)
format_axis(ax_pct)

for lbl in ax_pct.get_xticklabels():
    lbl.set_fontproperties(regular_font)
for lbl in ax_pct.get_yticklabels():
    lbl.set_fontproperties(regular_font)


# --- Deviations (differences in percentage points) ---
diff_living = y_living_pct - y_dwell_pct
diff_pop    = y_pop_pct    - y_dwell_pct

w2 = 0.15
bars_diff_liv = ax_diff.bar(
    x_pos - w2 / 2,
    diff_living,
    w2,
    color=PINK,
    edgecolor=BACKGROUND,
    linewidth=0.7,
    label="Living Area – Apartments",
)
bars_diff_pop = ax_diff.bar(
    x_pos + w2 / 2,
    diff_pop,
    w2,
    color=GREEN,
    edgecolor=BACKGROUND,
    linewidth=0.7,
    label="Population – Apartments",
)

ax_diff.set_title(
    "Percentage Deviation from Number of Apartments (in Percentage Points)",
    pad=10,
    fontproperties=bold_font,
)
ax_diff.set_ylabel(
    "Difference (percentage points)",
    labelpad=2,
    fontproperties=regular_font,
)
ax_diff.set_xticks(x_pos)
ax_diff.set_xticklabels(
    x,
    rotation=45,
    fontproperties=regular_font,
)
ax_diff.set_ylim(-1, 1)
ax_diff.axhline(0, color="#919191", linewidth=1.5, linestyle="--")

format_axis(ax_diff)
ax_diff.legend(
    loc="upper right",
    frameon=True,
    facecolor="white",
    prop=regular_font,
)
ax_diff.bar_label(
    bars_diff_liv,
    padding=3,
    fmt="%.2f",
    fontsize=8,
    fontproperties=regular_font,
)
ax_diff.bar_label(
    bars_diff_pop,
    padding=3,
    fmt="%.2f",
    fontsize=8,
    fontproperties=regular_font,
)

for lbl in ax_diff.get_xticklabels():
    lbl.set_fontproperties(regular_font)
for lbl in ax_diff.get_yticklabels():
    lbl.set_fontproperties(regular_font)


plt.subplots_adjust(hspace=0.7, wspace=0.25, top=0.9)

fig1.savefig(
    "../Plots/Bar.png",
    dpi=1000,
    bbox_inches="tight",
)

plt.show()


In [ ]:
# Vis 4: Waffle – Children / Retirees -----------------------------

period_of_interest = "2015–2019"

df_plot = (
    joined_df
    .query("five_year_period == @period_of_interest")
    .dropna(subset=["population_total"])
    .copy()
)

df_plot["num_children"] = df_plot["num_children"].fillna(0)
df_plot["num_retirees"] = df_plot["num_retirees"].fillna(0)

df_plot["other"] = (
    df_plot["population_total"]
    - df_plot["num_children"]
    - df_plot["num_retirees"]
).clip(lower=0)

df_plot["kreis_num"] = (
    df_plot["district_name"]
    .str.extract(r"(\d+)", expand=False)
    .astype(int)
)

df_plot = df_plot.sort_values("kreis_num")
districts = df_plot["district_name"].unique()

colors_waffle = [BLUE_LIGHT, PINK_LIGHT, "#E0E0E0"]

plots = {}
plot_index = 1
n = len(districts)
ncols = 4
nrows = math.ceil(n / ncols)

for district in districts:
    row = df_plot[df_plot["district_name"] == district].iloc[0]
    values = {
        "Children": int(row["num_children"]),
        "Retirees": int(row["num_retirees"]),
        "Others": int(row["other"]),
    }
    if sum(values.values()) == 0:
        continue

    plots[(nrows, ncols, plot_index)] = {"values": values}
    plot_index += 1

fig = plt.figure(
    FigureClass=Waffle,
    plots=plots,
    rows=10,
    columns=10,
    colors=colors_waffle,
    vertical=True,
    starting_location="SW",
    figsize=(15, 14),
)

fig.patch.set_facecolor(BACKGROUND)
fig.set_tight_layout(False)

fig.subplots_adjust(
    left=0,
    right=1,
    bottom=0.1,
    top=0.9,
    wspace=0.1,
    hspace=0.4,
)

fig.suptitle(
    f"Population Structure by District – {period_of_interest}",
    fontproperties=bold_font,
    fontsize=28,
    color=BLUE,
    y=1.03,
)

legend_handles = [
    Patch(color=colors_waffle[0], label="Children"),
    Patch(color=colors_waffle[1], label="Retirees"),
    Patch(color=colors_waffle[2], label="Others"),
]

fig.legend(
    handles=legend_handles,
    loc="upper center",
    ncol=3,
    frameon=False,
    bbox_to_anchor=(0.5, 1.0),
    prop=regular_font,
    fontsize=18,
)

for i, ax in enumerate(fig.axes):
    if i >= len(districts):
        break

    district_name = districts[i]
    row = df_plot[df_plot["district_name"] == district_name].iloc[0]
    total = row["population_total"]

    pct_kids = (row["num_children"] / total * 100) if total > 0 else 0
    pct_ret  = (row["num_retirees"] / total * 100) if total > 0 else 0

    # Remove Waffle's auto-legend
    ax.legend().remove()

    ax.text(
        x=0.0, y=1.24,
        s=district_name,
        fontproperties=bold_font,
        fontsize=18,
        color="#1a1a1a",
        ha="left",
        transform=ax.transAxes,
    )

    ax.text(
        x=0.0, y=1.13,
        s=f"{pct_kids:.1f}%",
        fontproperties=bold_font,
        fontsize=18,
        color=BLUE_LIGHT,
        ha="left",
        transform=ax.transAxes,
    )
    ax.text(
        x=0.27, y=1.13,
        s="Children",
        fontproperties=regular_font,
        fontsize=14,
        color="#666666",
        ha="left",
        transform=ax.transAxes,
    )

    ax.text(
        x=0.0, y=1.04,
        s=f"{pct_ret:.1f}%",
        fontproperties=bold_font,
        fontsize=18,
        color=PINK_LIGHT,
        ha="left",
        transform=ax.transAxes,
    )
    ax.text(
        x=0.27, y=1.04,
        s="Retirees",
        fontproperties=regular_font,
        fontsize=14,
        color="#666666",
        ha="left",
        transform=ax.transAxes,
    )

fig.savefig(
    "../Plots/Waffle.png",
    dpi=1000,
    bbox_inches="tight",
)

plt.show()


In [ ]:
# Vis 5: Bubble – Income vs Population vs New Buildings -------

from matplotlib import cm
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

df_all = bubble_df.copy()
df_all["year"] = df_all["year"].astype(int)

# -----------------------------------
# Fonts specifically for this chart
# -----------------------------------
title_font = bold_font.copy()
title_font.set_size(32)

axis_label_font = regular_font.copy()
axis_label_font.set_size(18)

tick_font = regular_font.copy()
tick_font.set_size(12)

legend_title_font = bold_font.copy()
legend_title_font.set_size(14)

legend_label_font = regular_font.copy()
legend_label_font.set_size(12)

bubble_label_font = bold_font.copy()
bubble_label_font.set_size(12)

# Base data
x_data = df_all["taxable_income_p50"]
y_data = df_all["population_total"]
size_data = df_all["num_new_buildings"].astype(float)
kreis_data = df_all["district_sort"]

# Bubble sizes
s_min = size_data.min()
s_max = size_data.max()

if size_data.nunique() <= 1 or s_max == s_min:
    sizes = np.full_like(size_data, 800.0)
else:
    sizes = 200 + (size_data - s_min) / (s_max - s_min) * (2500 - 200)

# Colors by district
unique_kreise = sorted(kreis_data.unique())
kreis_to_int = {k: i for i, k in enumerate(unique_kreise)}
cmap_kreise = cm.get_cmap("tab20", len(unique_kreise))
colors_by_kreis = kreis_data.map(kreis_to_int)

# Flag latest year per district
is_latest = df_all.groupby("district_sort")["year"].transform(
    lambda s: s == s.max()
)

# Figure & axis
fig, ax = plt.subplots(figsize=(12, 9))
fig.set_facecolor(BACKGROUND)
ax.set_facecolor(BACKGROUND)

# Older years
ax.scatter(
    x_data[~is_latest],
    y_data[~is_latest],
    s=sizes[~is_latest],
    c=colors_by_kreis[~is_latest],
    cmap=cmap_kreise,
    alpha=0.25,
    edgecolor="white",
    linewidth=0.8,
    zorder=2,
)

# Latest year
ax.scatter(
    x_data[is_latest],
    y_data[is_latest],
    s=sizes[is_latest],
    c=colors_by_kreis[is_latest],
    cmap=cmap_kreise,
    alpha=1.0,
    edgecolor="white",
    linewidth=1.2,
    zorder=4,
)

# District number labels inside bubbles (latest year only)
df_latest = df_all[is_latest]
for _, row in df_latest.iterrows():
    ax.text(
        row["taxable_income_p50"],
        row["population_total"],
        s=str(int(row["district_sort"])),
        ha="center",
        va="center",
        fontproperties=bubble_label_font,
        color="white",
        zorder=6,
    )

# Thousands formatter
def thousands(x, pos):
    try:
        return f"{int(x):,}".replace(",", "'")
    except (ValueError, TypeError):
        return ""

# Title & axes
ax.set_title(
    "Income vs Population by District",
    fontproperties=title_font,
    color="black",
    pad=35,
)

ax.set_xlabel(
    "Median taxable income",
    fontproperties=axis_label_font,
    labelpad=16,
)
ax.set_ylabel(
    "Total population",
    fontproperties=axis_label_font,
    labelpad=16,
)

# Y-axis formatting
ax.yaxis.set_major_formatter(mticker.FuncFormatter(thousands))

# Tick labels
ax.tick_params(axis="both", length=0, colors="black")
for lbl in ax.get_xticklabels():
    lbl.set_fontproperties(tick_font)
for lbl in ax.get_yticklabels():
    lbl.set_fontproperties(tick_font)

# Grid lines
ax.minorticks_on()
ax.grid(which="major", lw=0.8, alpha=0.5, color="#D6E6FF")
ax.grid(which="minor", lw=0.4, alpha=0.3, color="#D6E6FF")

# Legend for bubble size
valid_sizes = size_data[size_data > 0]

if len(valid_sizes) > 0:
    s_min_nonzero = valid_sizes.min()
    s_max_nonzero = valid_sizes.max()

    if np.isclose(s_min_nonzero, s_max_nonzero):
        size_raw = np.array([s_min_nonzero])
    else:
        size_raw = np.array([s_min_nonzero, s_max_nonzero])

    if s_max == s_min:
        marker_sizes = np.array([800.0]) * 0.6
    else:
        marker_sizes = 200 + (size_raw - s_min) / (s_max - s_min) * (2500 - 200)
        marker_sizes = marker_sizes * 0.6

    size_handles = [
        plt.scatter(
            [], [],
            s=ms,
            facecolor="none",
            edgecolor="black",
            linewidth=1.0,
        )
        for ms in marker_sizes
    ]

    size_labels = [
        f"{int(round(v)):,}".replace(",", "'") + " new buildings"
        for v in size_raw
    ]

    leg_size = ax.legend(
        handles=size_handles,
        labels=size_labels,
        title="Bubble size",
        prop=legend_label_font,
        loc="upper right",
        bbox_to_anchor=(1, 1),
        frameon=True,
        facecolor="white",
        edgecolor="lightgray",
        framealpha=1.0,
        borderpad=1.2,
        labelspacing=1.4,
        handlelength=2.0,
        handletextpad=1.0,
    )

    leg_size.get_title().set_fontproperties(legend_title_font)

# Spines
for spine in ax.spines.values():
    spine.set_visible(False)

plt.tight_layout()

fig.savefig(
    "../Plots/Bubble.png",
    dpi=300,
    bbox_inches="tight",
)

plt.show()
